In [1]:
#importer vos libs 
import warnings
warnings.simplefilter("ignore")
import pandas as pd 
import numpy as np 
import seaborn as sns
import sklearn as sklearn

In [2]:
df = pd.read_csv('../data/labeled_data.csv', sep=','); df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [13]:
df = df[['hate_speech','offensive_language','neither','class','tweet']]
df.shape

(24783, 5)

In [14]:
df['tweet'] = df['tweet'].apply(lambda tweet: re.sub('[^A-Za-z]+', ' ', tweet.lower()))


In [15]:
df.head()

,hate_speech,offensive_language,neither,class,tweet
0,0,0,3,2,rt mayasolovely as a woman you shouldn t comp...
1,0,3,0,1,rt mleew boy dats cold tyga dwn bad for cuffi...
2,0,3,0,1,rt urkindofbrand dawg rt sbaby life you ever ...
3,0,2,1,1,rt c g anderson viva based she look like a tr...
4,0,6,0,1,rt shenikaroberts the shit you hear about me ...


In [21]:
!pip install stop-words

In [23]:
!pip list | grep stop-words

stop-words          2018.7.23


In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from stop_words import get_stop_words


clf = make_pipeline(
    TfidfVectorizer(stop_words=get_stop_words('en')),
    OneVsRestClassifier(SVC(kernel='linear', probability=True))
)

clf = clf.fit(X=df['tweet'], y=df['class'])



In [26]:
text = "I hate you, please die!"
clf.predict_proba([text])[0]


array([0.64256281, 0.14180581, 0.21563137])

In [28]:
!pip install joblib

In [29]:
!pip list | grep joblib

joblib              1.0.1


In [31]:
import joblib
model_filename = "hatespeech.joblib.z"

joblib.dump((clf), model_filename)

['hatespeech.joblib.z']

In [32]:
clf1 = joblib.load("hatespeech.joblib.z")

In [35]:
text1 = "I love, good life!"
clf1.predict_proba([text1])[0]

array([0.01820455, 0.70288736, 0.27890809])

In [36]:
!pip install streamlit

Using legacy 'setup.py install' for blinker, since package 'wheel' is not installed.
    Running setup.py install for blinker: started
    Running setup.py install for blinker: finished with status 'done'


In [39]:
!pip list | grep streamlit

streamlit           0.80.0


In [ ]:
# Ci-dessous, code fait tout seul

In [4]:
import string
import re
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    #text = re.split('\W+', text_rc)    # tokenization     
    return text

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(df['tweet'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

24783 Number of tweets has 36456 words


In [10]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

,,a,aa,aaaaaaaaand,aaadontplayy,aaahhhhh,aahahah,aaliyah,aaliyahhhm,aaliyahhlovee,...,zulu,zvckslvtr,zwaaad,zwengersierra,zwhite,zwithr,zzachbarness,zzzentropy,zzzquil,zzzzzz
0,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y = df[['class']]
y.size

24783

In [8]:
df = df.drop(columns='class')
X = df
X.values

array([[0, 0, 3,
        "!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."],
       [0, 3, 0,
        '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!'],
       [0, 3, 0,
        '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit'],
       ...,
       [0, 3, 0,
        'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again'],
       [0, 6, 0, 'youu got wild bitches tellin you lies'],
       [0, 0, 3,
        '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB']],
      dtype=object)

In [ ]:
pipe.score(X, y)